<a href="https://colab.research.google.com/github/ProAek11/StrategyTradingFrendersis-AI-System/blob/main/RSI_EMA_Scalping_5min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5min RSI Scalping Strategy

In [ ]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_5_M_ASK_12.02.2019-12.02.2022.csv")
df.head()

,Local time,Open,High,Low,Close,Volume
0,12.02.2019 00:00:00.000 GMT+0200,1.12763,1.12796,1.12762,1.12780,53.71
1,12.02.2019 00:05:00.000 GMT+0200,1.12780,1.12785,1.12771,1.12783,36.31
2,12.02.2019 00:10:00.000 GMT+0200,1.12782,1.12790,1.12782,1.12787,29.00
3,12.02.2019 00:15:00.000 GMT+0200,1.12786,1.12788,1.12778,1.12779,81.81
4,12.02.2019 00:20:00.000 GMT+0200,1.12779,1.12787,1.12779,1.12786,91.52


In [ ]:
df=df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
224542,11.02.2022 23:35:00.000 GMT+0200,1.13461,1.13503,1.13459,1.13497,192.66
224543,11.02.2022 23:40:00.000 GMT+0200,1.13497,1.13498,1.13457,1.13472,126.31
224544,11.02.2022 23:45:00.000 GMT+0200,1.13472,1.13472,1.13452,1.13460,130.50
224545,11.02.2022 23:50:00.000 GMT+0200,1.13458,1.13495,1.13457,1.13487,278.16
224546,11.02.2022 23:55:00.000 GMT+0200,1.13485,1.13533,1.13470,1.13507,621.25


In [ ]:
import pandas_ta as ta
df["EMA"] = ta.ema(df.Close, length=50)
df["RSI"] = ta.rsi(df.Close, length=3)
a=ta.adx(df.High, df.Low, df.Close, length=5)
df['ADX']=a['ADX_5']
df['ATR']= df.ta.atr()
#help(ta.adx)

# EMA Trend Signal

In [ ]:
emasignal = [0]*len(df)
backcandles = 8

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMAsignal'] = emasignal

# RSI & ADX Signal

In [ ]:
RSIADXSignal = [0] * len(df)
for row in range(0, len(df)):
    RSIADXSignal[row] = 0
    if df.EMAsignal[row]==1 and df.RSI[row]>=80 and df.ADX[row]>=30:
        RSIADXSignal[row]=1
    if df.EMAsignal[row]==2 and df.RSI[row]<=20 and df.ADX[row]>=30:
        RSIADXSignal[row]=2

df['RSIADXSignal']=RSIADXSignal

# Candles Signal

In [ ]:
CandleSignal = [0] * len(df)
for row in range(1, len(df)):
    CandleSignal[row] = 0
    #if (RSIADXSignal[row]==1 or RSIADXSignal[row-1]==1) and (df.Open[row]>df.Close[row]):# and df.Close[row]<df.Close[row-1]):
    #    CandleSignal[row]=1
    #if (RSIADXSignal[row]==2 or RSIADXSignal[row-1]==2) and (df.Open[row]<df.Close[row]):# and df.Close[row]>df.Close[row-1]):
    #    CandleSignal[row]=2
    if RSIADXSignal[row-1]==1 and df.Open[row]>df.Close[row] and df.Close[row]<min(df.Close[row-1], df.Open[row-1]):
        CandleSignal[row]=1
    if RSIADXSignal[row-1]==2 and df.Open[row]<df.Close[row] and df.Close[row]>max(df.Close[row-1], df.Open[row-1]):
        CandleSignal[row]=2

df['TotSignal']=CandleSignal

# Visualizing Signals

In [ ]:
import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-4
    elif x['TotSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[3000:4000]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='red', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

NameError: name 'df' is not defined

In [ ]:
df.iloc[3030:3033]

,Local time,Open,High,Low,Close,Volume,EMA,RSI,ADX,ATR,EMAsignal,RSIADXSignal,TotSignal,pointpos
3030,26.02.2019 12:30:00.000 GMT+0200,1.13646,1.13660,1.13629,1.13629,1731.47,1.135933,20.650543,52.088617,0.000318,2,0,0,NaN
3031,26.02.2019 12:35:00.000 GMT+0200,1.13630,1.13639,1.13623,1.13624,1370.44,1.135945,17.305228,41.870703,0.000307,2,2,0,NaN
3032,26.02.2019 12:40:00.000 GMT+0200,1.13626,1.13641,1.13623,1.13634,1075.85,1.135960,44.350358,34.082804,0.000298,2,0,2,1.13613


## Stop Loss from Price

In [ ]:
SLSignal = [0] * len(df)
SLbackcandles = 3
for row in range(SLbackcandles, len(df)):
    mi=1e10
    ma=-1e10
    if df.TotSignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,df.High[i])
        SLSignal[row]=ma
    if df.TotSignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,df.Low[i])
        SLSignal[row]=mi

df['SLSignal']=SLSignal

## Backtesting Strategy

In [ ]:
dfpl = df[:]
def SIGNAL():
    return dfpl.TotSignal

In [ ]:
from backtesting import Strategy
from backtesting import Backtest
class MyStrat(Strategy):
    initsize = 0.02
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.3*self.data.ATR[-1]
        TPSLRatio = 1.3

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:47: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.



Loading BokehJS ...

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                  224546.0
Duration                             224546.0
Exposure Time [%]                    4.231185
Equity Final [$]                    97.686878
Equity Peak [$]                    100.739541
Return [%]                          -2.313122
Buy & Hold Return [%]                0.644618
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                    -3.22087
Avg. Drawdown [%]                   -0.137421
Max. Drawdown Duration               187942.0
Avg. Drawdown Duration            5994.324324
# Trades                               1170.0
Win Rate [%]                        40.940171
Best Trade [%]                       0.259678
Worst Trade [%]                      -0.23257
Avg. Trade [%]                    

In [ ]:
bt.plot(show_legend=False)

In [ ]:
dfpl = df[:]
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.02
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                  224546.0
Duration                             224546.0
Exposure Time [%]                    2.395044
Equity Final [$]                     97.01786
Equity Peak [$]                      100.1234
Return [%]                           -2.98214
Buy & Hold Return [%]                0.644618
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.450852
Avg. Drawdown [%]                   -1.237493
Max. Drawdown Duration               216785.0
Avg. Drawdown Duration           74551.666667
# Trades                               1214.0
Win Rate [%]                        46.540362
Best Trade [%]                       0.164641
Worst Trade [%]                     -0.188669
Avg. Trade [%]                    

In [ ]:
#bt.plot(show_legend=False)